In [24]:
url = input('URL을 입력하세요 : ')

import pandas as pd
from bs4 import BeautifulSoup
import glob
from selenium import webdriver
import requests 
import os
import time

driver = webdriver.Chrome('c:/chromedriver.exe')

# url = 'file:///C:/Users/Playdata/Desktop/shinhancard/shinhancard_20201110_900046432.html'
driver.get(url)
html = driver.page_source
soup = BeautifulSoup(html,'html.parser')

title_tag = soup.select('head > title')
title = title_tag[0].text

# 암호된 문서일 경우 if 구문으로 들어감
if title == '' :
    pw = '950730'
    driver.find_element_by_id('password').send_keys(pw)
    driver.find_element_by_xpath('//*[@id="aform"]/span/input').click()

time.sleep(5)
    
html = driver.page_source
soup = BeautifulSoup(html,'html.parser')

# 암호된 문서가 아닐 경우 아래있는 코드로 바로 들어감
tables = soup.select('div#email02 > table.tblList.cardUseList > tbody > tr')
tables_middleSum = soup.select('div#email02 > table.tblList.cardUseList > tbody > tr.middleSum')

for n in range(len(tables_middleSum )) :
    tables.remove(tables_middleSum[n])
    
# 명세서에서 추출한 데이터 string으로 변환하여 분리
seperate_tables = []                          
for n in range(len(tables)) :
    table = str(tables[n].text).split('\n')
    seperate_tables = seperate_tables + table

# seperate_tables에서 '사용날짜' 추출   
date_list = []
for n in range(1,len(seperate_tables), 12) :
    date = seperate_tables[n]
    date_list.append(date)

# seperate_tables에서 '사용처' 데이터만 추출    
store_list = []
for n in range(3,len(seperate_tables),12) :
    store = seperate_tables[n].replace(" ", "")
    store_list.append(store)

# 카카오 API활용하여 seperate_tables에서 '사용처 주소' 데이터만 추출    
location_list = []
def create_url(search_text):
    headers = {'Authorization': 'KakaoAK 3e8ea332f0cdef6293383ac7bf81a6fc'}

    base = "https://dapi.kakao.com/v2/local/search/keyword.json"
    param_query = "?query=" +search_text
#     print(f'base + param_query: {base + param_query}')
    respond = requests.get(base + param_query,headers=headers)
    return respond.json()

for n in range(len(store_list)) :
    store_name = store_list[n]
    result = create_url(store_name)
    tmp1 = result['documents']
    try :
        tmp2 = tmp1[0]
        tmp3 = tmp2['address_name']
        location_list.append(tmp3)
    except :
        tmp2 = '해당 주소가 없습니다.'
        location_list.append(tmp2)

# seperate_tables에서 '사용금액' 데이터만 추출  
price_list = [] 
for n in range(4,len(seperate_tables),12) :
    price = seperate_tables[n]
    price_list.append(price)

# 추출한 데이터로 DataFrame 변환
table_data = {'사용날짜':date_list, '사용처':store_list, '사용처주소':location_list,  '사용금액':price_list }
df1 = pd.DataFrame(table_data, columns=['사용날짜', '사용처', '사용처주소', '사용금액'])

driver.close()

df1

URL을 입력하세요file:///C:/Users/Playdata/Desktop/shinhancard/shinhancard_20200210_900046432.html


,사용날짜,사용처,사용처주소,사용금액
0,19.12.19,자판기,서울 용산구 이태원동 685,"1,000"
1,19.12.27,빽다방서울과기대점,서울 노원구 공릉동 441-1,"3,500"
2,19.12.27,모노태스크,해당 주소가 없습니다.,"30,415"
3,19.12.27,세븐일레븐노원공릉그린점,서울 노원구 공릉동 415-65,"1,000"
4,19.12.28,빽다방안암역점,서울 성북구 안암동5가 100-5,"2,000"
...,...,...,...,...
114,20.01.20,(주)티머니_지하철,해당 주소가 없습니다.,300
115,20.01.24,(주)티머니_지하철,해당 주소가 없습니다.,"1,250"
116,20.01.24,(주)티머니_지하철,해당 주소가 없습니다.,300
117,20.01.26,(주)티머니_지하철,해당 주소가 없습니다.,"1,250"


In [25]:
# DataFrame Excel로변환
folder = 'C:/Card'

if not os.path.exists(folder) :
    os.makedirs(folder)

tables_date = soup.select('body > div.wrap.check > div.content > div.receiveDate > span.date')
sheet_nm = str(tables_date[0].text)
excel_writer = pd.ExcelWriter('C:/Card/CardUseList.xlsx', 
                              engine='xlsxwriter')
df1.to_excel(excel_writer, index = False, sheet_name = sheet_nm )
excel_writer.save()